In [1]:
import numpy as np 
import pandas as pd 
import os 
import pickle 
import warnings
warnings.filterwarnings("ignore")

## Remove the empty files from the KSC MERLIN Cloud to Cloud Dataset

In [2]:
data_dir = '/Users/brandonmcclung/Data/KSC_Weather_Archive/MERLIN_CC/'
yrs = ['2017']
mos = ['01','02','03','04','05','06','07','08','09','10','11','12']
for yr in yrs:
    for mo in mos:
        dir = data_dir+yr+'_CC/'+mo+'/'
        files = os.listdir(dir)
        for file in files:
            try:
                    temp = pd.read_csv(dir+file,sep=' ',header=None,usecols=[0,2,6,8,12,15,20,22,24,26],low_memory=False)
            except:
                print('no data: '+file+' removing')
                os.remove(dir+file)


no data: KSCCC20170127.dat removing
no data: KSCCC20170126.dat removing
no data: KSCCC20170130.dat removing
no data: KSCCC20170124.dat removing
no data: KSCCC20170118.dat removing
no data: KSCCC20170119.dat removing
no data: KSCCC20170125.dat removing
no data: KSCCC20170131.dat removing
no data: KSCCC20170109.dat removing
no data: KSCCC20170121.dat removing
no data: KSCCC20170120.dat removing
no data: KSCCC20170112.dat removing
no data: KSCCC20170113.dat removing
no data: KSCCC20170111.dat removing
no data: KSCCC20170110.dat removing
no data: KSCCC20170128.dat removing
no data: KSCCC20170114.dat removing
no data: KSCCC20170101.dat removing
no data: KSCCC20170115.dat removing
no data: KSCCC20170129.dat removing
no data: KSCCC20170103.dat removing
no data: KSCCC20170117.dat removing
no data: KSCCC20170116.dat removing
no data: KSCCC20170102.dat removing
no data: KSCCC20170226.dat removing
no data: KSCCC20170225.dat removing
no data: KSCCC20170224.dat removing
no data: KSCCC20170220.dat r

## Build a dataframe for each month

In [2]:
data_dir = '/Users/brandonmcclung/Data/KSC_Weather_Archive/MERLIN_CC/'
yrs = ['2017']
mos = ['01','02','03','04','05','06','07','08','09','10','11','12']
i=0
for yr in yrs:
    for mo in mos:
        print(mo)
        if mo=='06':
            first_file=True
            dir = data_dir+yr+'_CC/'+mo+'/'
            files = os.listdir(dir)
            for file in files:
                if first_file==True:
                    first_file=False
                    merlin_cc = pd.read_csv(dir+file,sep=' ',header=None,usecols=[0,2,6,8,20,22,24,26],low_memory=False,\
                        on_bad_lines='skip',parse_dates={"Date_Time":[0,2]})
                else:
                    print(file)
                    merlin_cc = merlin_cc.append(pd.read_csv(dir+file,sep=' ',header=None,usecols=[0,2,6,8,20,22,24,26],\
                        on_bad_lines='skip',low_memory=False, parse_dates={"Date_Time":[0,2]}))
            data_stor = '/Users/brandonmcclung/Data/pickles/'
            file = open(data_stor+yr+'_'+mo+'_merlin_cc_df.p','wb')
            pickle.dump(merlin_cc,file)
            file.close()    
merlin_cc.set_index("Date_Time",inplace=True)
merlin_cc.rename({6:'Lat',8:'Lon',20:'SemiMajor',22:'SemiMinor',24:'Ellipse_Angle',26:'Sensors'},inplace=True,axis=1)
merlin_cc.dropna(inplace=True,axis=0)
#merlin_cc.columns=['Lat','Lon','SemiMajor','SemiMinor','EllipseAngle','Sensors']

01
02
03
04
05
06
KSCCC20170608.dat
KSCCC20170603.dat
KSCCC20170602.dat
KSCCC20170601.dat
KSCCC20170611.dat
KSCCC20170605.dat
KSCCC20170604.dat
KSCCC20170610.dat
KSCCC20170606.dat
KSCCC20170607.dat
07
08
09
10
11
12


In [47]:
def convert_lat_lon(cc_df):
    for i in range(len(cc_df['Lat'])):
        try:
            str_lat = cc_df['Lat'][i].split(':')
            str_lon = cc_df['Lon'][i].split(':')

            deg = float(str_lat[0])
            mi = float(str_lat[1])
            sec = float(str_lat[2])

            cc_df['Lat'][i] = deg+mi/60.+sec/3600.

            deg1 = float(str_lon[0])
            mi2 = float(str_lon[1])
            sec2 = float(str_lon[2])

            cc_df['Lon'][i] = deg1-mi2/60.-sec2/3600.
        except AttributeError:
            print(cc_df.index[i])
    return cc_df




In [22]:
import re
temp_merlin = merlin_cc

#temp_merlin['Lat_Sec']=''
#temp_merlin['Lat_Sec'] = temp_merlin['Lat'].apply(lambda x: re.findall(':(.+)', x)[0])
temp_merlin =  temp_merlin['Lat'].str.split(':',expand=True)
temp_merlin.columns = ['Lat_Deg','Lat_Min','Lat_Seconds']

temp_merlin['Lat_Deg'] = temp_merlin['Lat_Deg'].astype('float')
temp_merlin['Lat_Min'] = temp_merlin['Lat_Min'].astype('float')/60. 
temp_merlin['Lat_Seconds'] = temp_merlin['Lat_Seconds'].astype('float')/3600.
temp_merlin['Lat_Decimal'] = temp_merlin['Lat_Deg']+temp_merlin['Lat_Min']+temp_merlin['Lat_Seconds']



In [23]:
temp_merlin

,Lat_Deg,Lat_Min,Lat_Seconds,Lat_Decimal
Date_Time,,,,
2017-06-09 00:26:00.748000,27.0,0.750000,0.000000,27.750000
2017-06-09 04:02:29.010700,27.0,0.416667,0.014722,27.431389
2017-06-09 04:23:50.604700,27.0,0.450000,0.016111,27.466111
2017-06-09 21:42:16.458700,26.0,0.900000,0.009167,26.909167
2017-06-09 21:42:50.348700,26.0,0.783333,0.000278,26.783611
...,...,...,...,...
06/07/2017 22:18:45.5541000,26.0,0.716667,0.015556,26.732222
06/07/2017 22:18:46.3813000,26.0,0.983333,0.014444,26.997778
06/07/2017 22:22:49.2028000,26.0,0.933333,0.011944,26.945278
